<a href="https://colab.research.google.com/github/ihightower/my-first-binder/blob/main/pe084.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
# !pip install pandas

In [2]:
print('last saved 11nov 14:05')

last saved 11nov 14:05


In [3]:
import random
import pandas as pd
import time


In [4]:
# del logging, logger, ch, formatter
from importlib import reload
import logging
logging.shutdown()
reload(logging)


# create logger
logger = logging.getLogger('simple_example')
logger.setLevel(logging.DEBUG)

 # create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
#ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)


# Project Euler P084

<img src="https://projecteuler.net/project/images/p084_monopoly_board.png">




In the game, Monopoly, the standard board is set up in the following way:

A player starts on the GO square and adds the scores on two 6-sided dice to determine the number of squares they advance in a clockwise direction. Without any further rules we would expect to visit each square with equal probability: 2.5%. However, landing on G2J (Go To Jail), CC (community chest), and CH (chance) changes this distribution.

In addition to G2J, and one card from each of CC and CH, that orders the player to go directly to jail, **if a player rolls three consecutive doubles, they do not advance the result of their 3rd roll. Instead they proceed directly to jail.**

At the beginning of the game, the CC and CH cards are shuffled. When a player lands on CC or CH they take a card from the top of the respective pile and, after following the instructions, it is returned to the bottom of the pile. There are sixteen cards in each pile, but for the purpose of this problem we are only concerned with cards that order a movement; any instruction not concerned with movement will be ignored and the player will remain on the CC/CH square.


Community Chest (2/16 cards):
* Advance to GO
* Go to JAIL

Chance (10/16 cards):

* Advance to GO
* Go to JAIL
* Go to C1
* Go to E3
* Go to H2
* Go to R1
* Go to next R (railway company)
* Go to next R
* Go to next U (utility company)
* Go back 3 squares.


The heart of this problem concerns the likelihood of visiting a particular square. That is, the probability of finishing at that square after a roll. For this reason it should be clear that, with the exception of G2J for which the probability of finishing on it is zero, the CH squares will have the lowest probabilities, as 5/8 request a movement to another square, and it is the final square that the player finishes at on each roll that we are interested in. We shall make no distinction between "Just Visiting" and being sent to JAIL, and we shall also ignore the rule about requiring a double to "get out of jail", assuming that they pay to get out on their next turn.

By starting at GO and numbering the squares sequentially from 00 to 39 we can concatenate these two-digit numbers to produce strings that correspond with sets of squares.

Statistically it can be shown that the three most popular squares, in order, are JAIL (6.24%) = Square 10, E3 (3.18%) = Square 24, and GO (3.09%) = Square 00. So these three most popular squares can be listed with the six-digit modal string: 102400.

If, instead of using two 6-sided dice, two 4-sided dice are used, find the six-digit modal string.

# Code Starts

## Set Monopoly Squares

In [5]:
#pe084 - Monopoly odds
lst_monopoly_squares = [
    'GO', 'A1', 'CC1', 'A2', 'T1', 'R1', 'B1', 'CH1', 'B2', 'B3', 'JAIL', 'C1',
    'U1', 'C2', 'C3', 'R2', 'D1', 'CC2', 'D2', 'D3', 'FP', 'E1', 'CH2', 'E2',
    'E3', 'R3', 'F1', 'F2', 'U2', 'F3', 'G2J', 'G1', 'G2', 'CC3', 'G3', 'R4',
    'CH3', 'H1', 'T2', 'H2'
]

lst_comchest = [0, 10] + [99] * 14
random.shuffle(lst_comchest)

lst_chance = [0, 10, 11, 24, 39, 5, [5, 15, 25, 35], [5, 15, 25, 35], [12, 28], -3] + [99] * 6
random.shuffle(lst_chance)

print(lst_comchest)
print(lst_chance)



[99, 99, 10, 99, 99, 0, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99]
[99, 5, 24, -3, 99, 99, 0, 99, [5, 15, 25, 35], 39, [12, 28], 99, [5, 15, 25, 35], 99, 11, 10]


In [6]:
lst_comchest_pos = [2, 17, 33]
lst_chance_pos = [7, 22, 36]

In [7]:
print('comechest cards: ', lst_comchest)
print('chance cards: ',lst_chance)

comechest cards:  [99, 99, 10, 99, 99, 0, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99]
chance cards:  [99, 5, 24, -3, 99, 99, 0, 99, [5, 15, 25, 35], 39, [12, 28], 99, [5, 15, 25, 35], 99, 11, 10]


## Functions

### func: move_from

In [35]:
def move_from(currpos):
    
    global gRollingDoubles
    
    dice_sides = 6
    
    dice1 = random.randint(1, dice_sides)
    dice2 = random.randint(1, dice_sides)
    
    dice_roll = dice1 + dice2
    
    if dice1 == dice2:
        gRollingDoubles += 1
    else:
        gRollingDoubles = 0
 
    if gRollingDoubles == 1:
        logger.debug(f'       rolled 1st double in {currpos}')
        # logger.setLevel(logging.WARNING)
        pass
    
    if gRollingDoubles == 2:
        logger.warning(f'       rolled 2nd double in {currpos}')
        # logger.setLevel(logging.WARNING)
        pass
    
    if gRollingDoubles == 3:
        # logger.setLevel(logging.DEBUG)
        
        logger.warning(f'       rolled 3rd double in {currpos}... and now going to jail')
 #         print('rolled 3 doubles, going to jail.. from ', newpos)
        newpos = 10 # go to jail
        gRollingDoubles = 0
        
        logger.warning(f'...... dice rolled: {dice1}+{dice2}={dice_roll}, from: {currpos} to: {newpos}')
        return newpos

    
    newpos = (currpos + dice_roll) % 40
 
 
    logger.debug(f'...... dice rolled: {dice1}+{dice2}={dice_roll}, from: {currpos} to: {newpos}')
    
    
    if newpos == 20:
        logger.debug(f'in {newpos}... should go to jail. RollingDouble#:{gRollingDoubles}')
        newpos = 10
        #gRollingDoubles = 0


    

 #     print('\n...... dice rolled', currpos, dice_roll, newpos)
    return newpos



```
...... dice rolled: 6, from: 25 to: 31
...... dice rolled: 9, from: 31 to: 0
rolled 1st double in 0
...... dice rolled: 10, from: 0 to: 10
...... dice rolled: 10, from: 10 to: 10
```



### func: take_chance_card

In [9]:
# chace cards:  [11, -3, [12, 28], 99, 99, [5, 15, 25, 35], 99, 0, [5, 15, 25, 35], 39, 99, 99, 24, 5, 10, 99]
def take_chance_card(currpos):
    global gCHANCELOC
    newpos = currpos
    
    chance_card = lst_chance[gCHANCELOC]
    logger.info(f'  in pos:{currpos}, seq: {gCHANCELOC}, card: {chance_card} {type(chance_card)}')
 #     print('  in pos:', currpos, ' seq: ', gCHANCELOC, ' card: ',  chance_card, type(chance_card))

    if (chance_card == 99):
        newpos = currpos
        pass
 #         print(' STAYING PUT RIGHT HERE')
    elif (type(chance_card) is int and chance_card < 0):
        newpos = currpos + chance_card
        newpos %= 40
        
 #         print('>>> moving to: ', newpos)
        
    elif (type(chance_card) is int):   
        newpos = chance_card
 #         print('>>> moving to: ', newpos)
        
    else:
 #         print('>>> moving to: ', 'sorry i am a list, i think')
                
        if currpos > chance_card[-1]:
            newpos = chance_card[0]
        else:    
            for i in chance_card:
                if currpos > i:
                    continue
                else:
                    newpos = i
                    break
        
    
    
    gCHANCELOC += 1
    gCHANCELOC %= len(lst_chance)
    
    return newpos
    

### func: take_comchest_card

In [10]:
#@title
# comechest cards:  [99, 99, 99, 0, 99, 99, 99, 99, 99, 10, 99, 99, 99, 99, 99, 99]

def take_comchest_card(currpos):
    global gCOMCHESTLOC
    newpos = currpos
    
    comchest_card = lst_comchest[gCOMCHESTLOC]
    logger.info(f'  in pos:{currpos}, seq: {gCOMCHESTLOC}, card: {comchest_card} {type(comchest_card)}')
  #     print('  in pos:', currpos, ' seq: ', gCOMCHESTLOC, ' card: ', comchest_card, type(comchest_card))
    
    if (comchest_card == 99):
        newpos = currpos
        pass
 #         print(' STAYING PUT RIGHT HERE')
    elif (type(comchest_card) is int):
        newpos = comchest_card
 #         print('>>> moving to: ', newpos)
        
    else:
        pass
 #         print('>>> moving to: ', 'sorry i am a list, i think')

        
        
        if currpos > comchest_card[-1]:
            newpos = comchest_card[0]
        else:    
            for i in comchest_card:
                if currpos > i:
                    continue
                else:
                    newpos = i
                    break        
        
    gCOMCHESTLOC += 1
    gCOMCHESTLOC %= len(lst_comchest)
    
    return newpos

### func: run_simulation

In [44]:
def run_simulation(simcount, currpos=0):

  for simu in range(simcount):
      # time.sleep(1)

      currpos = move_from(currpos)
      
  #     print(' in position now:', currpos, lst_monopoly_squares[currpos])
      
      if currpos in lst_chance_pos:
          
          # logger.setLevel(logging.CRITICAL)
          
          logger.info('\n>>> i am taking chance')
  #         print('\n>>> i am taking chance')
          currpos = take_chance_card(currpos)
          logger.info(f'   moved to {currpos}\n')
          
          # logger.setLevel(logging.INFO)
  #         print('.... moved to', currpos)
          pass
          #print(mychance_card)
      
      if currpos in lst_comchest_pos:
          # logger.setLevel(logging.CRITICAL)
          # logger.setLevel(logging.INFO)
          logger.info('\n>>> i am taking community chest')
  #         print('\n>>> i am community chest')
          currpos = take_comchest_card(currpos)
          logger.info(f'   moved to {currpos}\n')
          # logger.setLevel(logging.INFO)

  #         print('.... moved to', currpos)
          #print(mycomchest_card)
          
      if gRollingDoubles == 0 or True:
        logger.info(f'yes move counted {currpos}')
        dictsq[currpos] = dictsq.get(currpos, 0) + 1

  #     print(dice2, cuminsquare, insquare)

  print(dict(sorted(dictsq.items())))

  data_items = dictsq.items()
  data_list = list(data_items)

  df = pd.DataFrame(data_list)
  return df
  #df.sort_values([1], ascending=False)

 

# Run Simulation

<a href="https://projecteuler.net/problem=84">problem link</a>

In [46]:
 # answer: 102400

currpos = 0 #helllo
dictsq = {}

gCOMCHESTLOC = 0
gCHANCELOC = 0
gRollingDoubles = 0

logger.setLevel(logging.DEBUG)
# logger.setLevel(logging.WARNING)
# logger.setLevel(logging.CRITICAL)

simcount = 1_000#_000
df = run_simulation(simcount)


df['pct'] = df.iloc[:,1]/simcount *100
df.sort_values([1], ascending=False)

       rolled 1st double in 0
...... dice rolled: 6+6=12, from: 0 to: 12
yes move counted 12
       rolled 2nd double in 12
...... dice rolled: 2+2=4, from: 12 to: 16
yes move counted 16
...... dice rolled: 6+4=10, from: 16 to: 26
yes move counted 26
...... dice rolled: 5+6=11, from: 26 to: 37
yes move counted 37
...... dice rolled: 1+3=4, from: 37 to: 1
yes move counted 1
...... dice rolled: 6+2=8, from: 1 to: 9
yes move counted 9
...... dice rolled: 5+3=8, from: 9 to: 17

>>> i am taking community chest
  in pos:17, seq: 0, card: 99 <class 'int'>
   moved to 17

yes move counted 17
...... dice rolled: 6+1=7, from: 17 to: 24
yes move counted 24
       rolled 1st double in 24
...... dice rolled: 2+2=4, from: 24 to: 28
yes move counted 28
...... dice rolled: 2+4=6, from: 28 to: 34
yes move counted 34
...... dice rolled: 1+5=6, from: 34 to: 0
yes move counted 0
...... dice rolled: 5+4=9, from: 0 to: 9
yes move counted 9
       rolled 1st double in 9
...... dice rolled: 1+1=2, from: 9 to:

{0: 28, 1: 16, 2: 22, 3: 22, 4: 29, 5: 27, 6: 21, 7: 13, 8: 30, 9: 24, 10: 48, 11: 30, 12: 26, 13: 24, 14: 29, 15: 24, 16: 28, 17: 29, 18: 33, 19: 36, 21: 26, 22: 9, 23: 30, 24: 34, 25: 30, 26: 27, 27: 29, 28: 15, 29: 26, 30: 18, 31: 31, 32: 29, 33: 21, 34: 23, 35: 16, 36: 7, 37: 31, 38: 24, 39: 35}


,0,1,pct
28,10,48,4.8
12,19,36,3.6
32,39,35,3.5
7,24,34,3.4
21,18,33,3.3
3,37,31,3.1
35,31,31,3.1
19,8,30,3.0
37,23,30,3.0
27,25,30,3.0


In [13]:
1/0

ZeroDivisionError: ignored

<a id="section-one"></a>
## Helper Functions

**^html links hidden here, fork notebook then click this cell to see it**
<a id="section-two"></a>
## Section Two
**^html links hidden here, fork notebook then click this cell to see it**


In [ ]:
import re

lst1 = ['JAIL', 'C1', 'E3', 'H2', 'R1']

for x in lst1:    
    print(x, lst_monopoly_squares.index(x))

lst2 = ['^R', '^U', '^CC', '^CH']

for x in lst2:    
    r = re.compile(x)
    new_list = list(filter(r.match, lst_monopoly_squares))
    print(new_list)
    
    for y in new_list:
        print(x, lst_monopoly_squares.index(y))

## Image Testing (Works!)

In [ ]:
# Importing the PIL library
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

In [ ]:
import requests
from io import BytesIO
url = "https://projecteuler.net/project/images/p084_monopoly_board.png"
response = requests.get(url)
orig_img = Image.open(BytesIO(response.content))

In [ ]:
# Open an Image
img = orig_img.copy()

myfont = ImageFont.truetype(r'/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf', 20)

# myfont = ImageFont.truetype("arial.ttf", 1)
 
# Call draw Method to add 2D graphics in an image
I1 = ImageDraw.Draw(img) 

mysq = 0

for i in range(4):
  
  
  if i == 0:
    print ('walk right 0 to 9')
    for sq in range(10):
      x = 40.7*sq + 10
      y = 37.8*i + 5

      m = lst_monopoly_squares[mysq]
      mysq+=1

      
      I1.text((x, y),f'{m}', font=myfont, fill =(255, 0, 0))

  if i == 1:
    print ('walk down 0 to 9')
   
    for sq in range(10):
      x = 40.7*10 + 10
      y = 37.8*sq + 5
      m = lst_monopoly_squares[mysq]
      mysq+=1      
      I1.text((x, y),f'{m}',font=myfont, fill =(255, 0, 0))

  if i == 2:
    print ('walk left 0 to 9')

    for sq in range(10, 0, -1):
      x = 40.7*sq + 10
      y = 37.8*10 #+ 5
      m = lst_monopoly_squares[mysq]
      mysq+=1      
      I1.text((x, y),f'{m}', font=myfont,fill =(255, 0, 0))
  
  if i == 3:
    print ('walk up 0 to 9')

    for sq in range(10, 0, -1):
      x = 40.7*0 + 10
      y = 37.8*sq + 3
      m = lst_monopoly_squares[mysq]
      mysq+=1      
      I1.text((x, y),f'{m}', font=myfont,fill =(255, 0, 0))

 # Display edited image
img

In [ ]:
# Open an Image
img = orig_img.copy()
 
# Call draw Method to add 2D graphics in an image
I1 = ImageDraw.Draw(img)

for sq in range(40):
  
  x = 40.7*i + 10
  for j in range(11):
    if (i >=1 and i <= 9) and (j>=1 and j<=9):
      continue
    
  
    y = 37.8*j + 5

    I1.text((x, y), f'{i},{j}', fill =(255, 0, 0))
  
# Display edited image
img


In [ ]:
# Open an Image
img = orig_img.copy()
 
# Call draw Method to add 2D graphics in an image
I1 = ImageDraw.Draw(img)
 
# Custom font style and font size
# myFont = ImageFont.truetype('FreeMono.ttf', 65)
 
# Add Text to an image

for i in range(11):
  x = 40.7*i + 10
  for j in range(11):
    if (i >=1 and i <= 9) and (j>=1 and j<=9):
      continue
    
  
    y = 37.8*j + 5

    I1.text((x, y), f'{i},{j}', fill =(255, 0, 0))
  
# Display edited image
img


## Simple Test1

In [ ]:
mylist = [5, 15, 25, 35]

currno = 36

if currno > mylist[-1]:
    selection = mylist[0]
else:    
    for i in mylist:
        if currno > i:
            continue
        else:
            selection = i
            break

print(selection)
    

## Test Markdown

Clicking on each link jumps you to the section that is marked. It is really intuitive to use! 
I would recommend adding a table of contents to any of your published notebooks, so that the reader doesn't get lost.
You would want everyone to enjoy reading that kernel that you spent so long on!

Links use HTML to target your browser to that specific section of your Kaggle notebook.
To see the inner workings and code, please fork this notebook and click on the cells to see the markdown code

* [Introduction](#section-one)
* [Body](#section-two)
    - [Subsectionaaa 1](#subsection-one)
    - [Subsectionaaa 2](#anything-you-like)
* [Conclusionaaa](#section-three)

## Logging Test

In [ ]:
# 'application' code
logger.debug('debug message')
logger.info('info message')
logger.info('warn message')
logger.error('error message')
logger.critical('critical message')